In [1]:
import sagemaker

session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [2]:
%store -r model_name
model_name

'model-VWXTC'

In [3]:
import boto3
client = boto3.client('sagemaker')

In [4]:
response = client.describe_model(ModelName=model_name)
response

{'ModelName': 'model-VWXTC',
 'PrimaryContainer': {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-06-13-17-34-37-316/output/model.tar.gz',
  'Environment': {}},
 'ExecutionRoleArn': 'arn:aws:iam::581320662326:role/SuperAdminRole',
 'CreationTime': datetime.datetime(2021, 6, 13, 17, 38, 50, 616000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/model-vwxtc',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': 'b791a0d4-e64c-4787-b1f1-600e0077ad9c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b791a0d4-e64c-4787-b1f1-600e0077ad9c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '484',
   'date': 'Sun, 13 Jun 2021 18:20:05 GMT'},
  'RetryAttempts': 0}}

In [5]:
container = response['PrimaryContainer']['Image']
container

'683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-1'

In [6]:
model_data = response['PrimaryContainer']['ModelDataUrl']
model_data

's3://sagemaker-us-east-1-581320662326/sagemaker-xgboost-2021-06-13-17-34-37-316/output/model.tar.gz'

In [7]:
model = sagemaker.model.Model(
    name=model_name,
    image_uri=container,
    model_data=model_data,
    role=role,
    sagemaker_session=session
)

In [8]:
%%time

model.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.m5.large'
)

Using already existing model: model-VWXTC


---------------!CPU times: user 259 ms, sys: 11.1 ms, total: 270 ms
Wall time: 7min 32s


In [9]:
model.endpoint_name

'model-VWXTC-2021-06-13-18-20-15-736'

In [10]:
from sagemaker import Predictor
predictor = Predictor(endpoint_name=model.endpoint_name)

In [12]:
%store -r s3_bucket_name
%store -r prefix
base = f"s3://{s3_bucket_name}/{prefix}"
s3_capture_upload_path = f"{base}/model-monitor"

In [13]:
from sagemaker.model_monitor import DataCaptureConfig

In [14]:
data_capture_config = DataCaptureConfig(
    enable_capture = True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
    kms_key_id=None,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json"]
)

In [15]:
%%time

predictor.update_data_capture_config(
    data_capture_config=data_capture_config
)

-----------------!CPU times: user 257 ms, sys: 16.6 ms, total: 273 ms
Wall time: 8min 33s


In [16]:
boto_session = boto3.Session(region_name=region)
runtime = boto_session.client(
    service_name='sagemaker-featurestore-runtime', 
    region_name=region
)

In [17]:
feature_group_name = 'cookbook-feature-group'

record_response = runtime.get_record(
    FeatureGroupName=feature_group_name, 
    RecordIdentifierValueAsString="950"
)

In [18]:
record_response['Record']

[{'FeatureName': 'approved', 'ValueAsString': '1'},
 {'FeatureName': 'sex', 'ValueAsString': '1'},
 {'FeatureName': 'math', 'ValueAsString': '92'},
 {'FeatureName': 'science', 'ValueAsString': '83'},
 {'FeatureName': 'technology', 'ValueAsString': '86'},
 {'FeatureName': 'random1', 'ValueAsString': '96'},
 {'FeatureName': 'random2', 'ValueAsString': '67'},
 {'FeatureName': 'index', 'ValueAsString': '950'},
 {'FeatureName': 'event_time', 'ValueAsString': '1623604510.0'}]

In [19]:
test_record_list = [
    record_response['Record'][1]['ValueAsString'],
    record_response['Record'][2]['ValueAsString'],
    record_response['Record'][3]['ValueAsString'],
    record_response['Record'][4]['ValueAsString'],
    record_response['Record'][5]['ValueAsString'],
    record_response['Record'][6]['ValueAsString'],
]

test_record_list

['1', '92', '83', '86', '96', '67']

In [20]:
csv_input = ','.join(test_record_list)
csv_input

'1,92,83,86,96,67'

In [21]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [22]:
predictor.predict(csv_input)

0.9916712045669556

In [23]:
endpoint_name = predictor.endpoint_name

%store endpoint_name
endpoint_name

Stored 'endpoint_name' (str)


'model-VWXTC-2021-06-13-18-20-15-736'

In [24]:
%store csv_input

Stored 'csv_input' (str)
